# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Union

In [ ]:
# --- Funções para criar Matrizes de Transformação Homogênea (3x3) ---

def matriz_translacao(vetor: Union[List, np.ndarray]) -> np.ndarray:
    """Cria uma matriz de translação 3x3."""
    dx, dy = vetor
    return np.array([
        [1, 0, dx],
        [0, 1, dy],
        [0, 0, 1]
    ])

def matriz_escala(fatores: Union[float, List, np.ndarray], origem: Tuple[float, float] = (0, 0)) -> np.ndarray:
    """Cria uma matriz de escala 3x3 em relação a uma origem."""
    sx, sy = fatores if isinstance(fatores, (list, np.ndarray)) else (fatores, fatores)
    ox, oy = origem
    return matriz_translacao([ox, oy]) @ np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]]) @ matriz_translacao([-ox, -oy])

def matriz_rotacao(angulo_graus: float, origem: Tuple[float, float] = (0, 0)) -> np.ndarray:
    """Cria uma matriz de rotação 3x3 em relação a uma origem."""
    angulo_rad = np.radians(angulo_graus)
    c, s = np.cos(angulo_rad), np.sin(angulo_rad)
    ox, oy = origem
    return matriz_translacao([ox, oy]) @ np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]]) @ matriz_translacao([-ox, -oy])

def matriz_reflexao(eixo: str = 'y') -> np.ndarray:
    """Cria uma matriz de reflexão 3x3."""
    if eixo == 'y': return np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
    if eixo == 'x': return np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])
    raise ValueError("O eixo deve ser 'x' ou 'y'")

def matriz_cisalhamento(k: float, direcao: str = 'horizontal') -> np.ndarray:
    """Cria uma matriz de cisalhamento 3x3."""
    if direcao == 'horizontal': return np.array([[1, k, 0], [0, 1, 0], [0, 0, 1]])
    if direcao == 'vertical': return np.array([[1, 0, 0], [k, 1, 0], [0, 0, 1]])
    raise ValueError("A direção deve ser 'horizontal' ou 'vertical'")

In [ ]:
# --- Classe Principal Refatorada ---

class FormaGeometrica:
    def __init__(self, pontos: np.ndarray, nome: str = "Forma"):
        """Inicializa a forma e converte os pontos para coordenadas homogêneas."""
        pontos_2d = np.atleast_2d(pontos)
        self.pontos_homogeneos = np.hstack([pontos_2d, np.ones((pontos_2d.shape[0], 1))])
        self.nome = nome

    @property
    def pontos(self) -> np.ndarray:
        """Retorna os pontos em coordenadas 2D para uso externo."""
        return self.pontos_homogeneos[:, :2]

    def __repr__(self) -> str:
        return f"{self.nome}(pontos=\n{self.pontos})"

    def aplicar_matriz(self, matriz: np.ndarray, novo_nome: str) -> 'FormaGeometrica':
        """Aplica uma matriz de transformação aos pontos da forma."""
        novos_pontos_homogeneos = (matriz @ self.pontos_homogeneos.T).T
        return FormaGeometrica(novos_pontos_homogeneos[:, :2], novo_nome)

    def transladar(self, vetor: Union[List, np.ndarray]) -> 'FormaGeometrica':
        return self.aplicar_matriz(matriz_translacao(vetor), f"{self.nome} transladado")

    def escalar(self, fatores: Union[float, List, np.ndarray], origem: Tuple[float, float] = (0, 0)) -> 'FormaGeometrica':
        return self.aplicar_matriz(matriz_escala(fatores, origem), f"{self.nome} escalado")

    def rotacionar(self, angulo_graus: float, origem: Tuple[float, float] = (0, 0)) -> 'FormaGeometrica':
        return self.aplicar_matriz(matriz_rotacao(angulo_graus, origem), f"{self.nome} rotacionado")

    def refletir(self, eixo: str = 'y') -> 'FormaGeometrica':
        return self.aplicar_matriz(matriz_reflexao(eixo), f"{self.nome} refletido")

    def cisalhar(self, k: float, direcao: str = 'horizontal') -> 'FormaGeometrica':
        return self.aplicar_matriz(matriz_cisalhamento(k, direcao), f"{self.nome} cisalhado")


In [ ]:
# --- Funções de Plotagem ---

def configurar_plot(ax, titulo: str):
    """Aplica configurações comuns a um eixo de plotagem."""
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(titulo, fontsize=14)
    ax.set_xlabel("Eixo X")
    ax.set_ylabel("Eixo Y")

def plotar_transformacao(original: FormaGeometrica, transformada: FormaGeometrica, titulo: str):
    """Plota uma transformação simples (original vs. final)."""
    fig, ax = plt.subplots(figsize=(8, 8))

    # Plotar original
    if original.pontos.shape[0] > 1:
        pontos_plot = np.vstack([original.pontos, original.pontos[0]])
        ax.plot(pontos_plot[:, 0], pontos_plot[:, 1], 'o-', label="Original", color='blue')
    else:
        ax.scatter(original.pontos[:, 0], original.pontos[:, 1], s=100, label="Original", color='blue', zorder=5)

    # Plotar transformada
    if transformada.pontos.shape[0] > 1:
        pontos_plot = np.vstack([transformada.pontos, transformada.pontos[0]])
        ax.plot(pontos_plot[:, 0], pontos_plot[:, 1], 'o--', label="Transformada", color='red')
    else:
        ax.scatter(transformada.pontos[:, 0], transformada.pontos[:, 1], s=100, label="Transformada", color='red', zorder=5)

    configurar_plot(ax, titulo)
    ax.legend()
    plt.show() # Garante que o gráfico seja exibido

def plotar_transformacao_composta(titulo: str, passos: List[FormaGeometrica]):
    """Plota uma sequência de transformações, mostrando cada passo."""
    fig, ax = plt.subplots(figsize=(8, 8))
    cores = plt.cm.viridis(np.linspace(0, 1, len(passos)))
    estilos = ['o-', 'o--', 'o-.', 'o:']

    for i, forma in enumerate(passos):
        label = f"Passo {i}: {forma.nome}" if i > 0 else "Original"
        cor = cores[i]
        estilo = estilos[i % len(estilos)]

        if forma.pontos.shape[0] > 1:
            pontos_plot = np.vstack([forma.pontos, forma.pontos[0]])
            ax.plot(pontos_plot[:, 0], pontos_plot[:, 1], estilo, label=label, color=cor, zorder=i+5)
        else:
            ax.scatter(forma.pontos[:, 0], forma.pontos[:, 1], s=100+i*20, label=label, color=cor, zorder=i+5)

    configurar_plot(ax, titulo)
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    fig.tight_layout()
    plt.show() # Garante que o gráfico seja exibido

In [ ]:
# --- Exercício 1: Translação Simples ---
p1 = FormaGeometrica(np.array([2, 3]), "P")
p1_transf = p1.transladar([4, -2])
print(f"--- Exercício 1 ---\nOriginal: {p1.pontos}\nFinal: {p1_transf.pontos}\n")
plotar_transformacao(p1, p1_transf, "Exercício 1: Translação Simples")

In [ ]:
# --- Exercício 2: Escala Uniforme ---
tri2 = FormaGeometrica(np.array([[1, 1], [3, 1], [2, 4]]), "Triângulo")
tri2_transf = tri2.escalar(2)
print(f"--- Exercício 2 ---\nOriginal: \n{tri2.pontos}\nFinal: \n{tri2_transf.pontos}\n")
plotar_transformacao(tri2, tri2_transf, "Exercício 2: Escala Uniforme")


In [ ]:
# --- Exercício 3: Escala Não Uniforme ---
tri3 = FormaGeometrica(np.array([[1, 1], [3, 1], [2, 4]]), "Triângulo")
tri3_transf = tri3.escalar([2, 0.5])
print(f"--- Exercício 3 ---\nOriginal: \n{tri3.pontos}\nFinal: \n{tri3_transf.pontos}\n")
plotar_transformacao(tri3, tri3_transf, "Exercício 3: Escala Não Uniforme")

In [ ]:
# --- Exercício 4: Rotação em Torno da Origem ---
p4 = FormaGeometrica(np.array([1, 0]), "P")
p4_transf = p4.rotacionar(90) # Sentido anti-horário
print(f"--- Exercício 4 ---\nOriginal: {p4.pontos}\nFinal: {np.round(p4_transf.pontos, 5)}\n")
plotar_transformacao(p4, p4_transf, "Exercício 4: Rotação 90° Anti-horário")

In [ ]:
# --- Exercício 5: Rotação de um Polígono ---
quad5 = FormaGeometrica(np.array([[1, 1], [1, 4], [4, 4], [4, 1]]), "Quadrado")
quad5_transf = quad5.rotacionar(-45) # Sentido horário
print(f"--- Exercício 5 ---\nOriginal: \n{quad5.pontos}\nFinal: \n{np.round(quad5_transf.pontos, 5)}\n")
plotar_transformacao(quad5, quad5_transf, "Exercício 5: Rotação 45° Horário")

In [ ]:
# --- Exercício 6: Reflexão Simples ---
p6 = FormaGeometrica(np.array([2, 5]), "P")
p6_transf = p6.refletir(eixo='y')
print(f"--- Exercício 6 ---\nOriginal: {p6.pontos}\nFinal: {p6_transf.pontos}\n")
plotar_transformacao(p6, p6_transf, "Exercício 6: Reflexão no Eixo Y")

In [ ]:
# --- Exercício 7: Reflexão de um Triângulo ---
tri7 = FormaGeometrica(np.array([[2, 3], [4, 3], [3, 5]]), "Triângulo")
tri7_transf = tri7.refletir(eixo='x')
print(f"--- Exercício 7 ---\nOriginal: \n{tri7.pontos}\nFinal: \n{tri7_transf.pontos}\n")
plotar_transformacao(tri7, tri7_transf, "Exercício 7: Reflexão no Eixo X")

In [ ]:
# --- Exercício 8: Cisalhamento Horizontal ---
p8 = FormaGeometrica(np.array([2, 3]), "P")
p8_transf = p8.cisalhar(k=2)
print(f"--- Exercício 8 ---\nOriginal: {p8.pontos}\nFinal: {p8_transf.pontos}\n")
plotar_transformacao(p8, p8_transf, "Exercício 8: Cisalhamento Horizontal (k=2)")


In [ ]:
# --- Exercício 9: Composição de Transformações ---
p9 = FormaGeometrica(np.array([3, 2]), "P")
p9_t1 = p9.transladar([1, -1])
p9_t2 = p9_t1.rotacionar(90)
p9_t3 = p9_t2.escalar(2)
print(f"--- Exercício 9 ---\nOriginal: {p9.pontos}\nPasso 1: {p9_t1.pontos}"
      f"\nPasso 2: {np.round(p9_t2.pontos, 5)}\nFinal: {np.round(p9_t3.pontos, 5)}\n")
plotar_transformacao_composta("Exercício 9: Composição de Transformações", [p9, p9_t1, p9_t2, p9_t3])


In [ ]:
# --- Exercício 10: Combinação de Transformações ---
ret10 = FormaGeometrica(np.array([[1, 1], [5, 1], [5, 3], [1, 3]]), "Retângulo")
matriz_final = matriz_reflexao('y') @ matriz_escala([1.5, 0.5]) @ matriz_translacao([-2, 3])
ret10_final = ret10.aplicar_matriz(matriz_final, "Final")
print(f"--- Exercício 10 ---\nOriginal: \n{ret10.pontos}\nFinal: \n{np.round(ret10_final.pontos, 5)}\n")

# Para visualização dos passos intermediários:
ret10_t1 = ret10.transladar([-2, 3])
ret10_t2 = ret10_t1.escalar([1.5, 0.5])
ret10_t3 = ret10_t2.refletir(eixo='y')
plotar_transformacao_composta("Exercício 10: Combinação de Transformações", [ret10, ret10_t1, ret10_t2, ret10_t3])
